# Redis Advanced Features

Utilisation des fonctionnalités avancées de Redis (RediSearch, etc.)

In [ ]:
from utils.redis_client import RedisClient

client = RedisClient()

# Recherche de films par genre
action_movies = client.search_movies("@genre:{Action}")
print("Action movies:", action_movies)